In [ ]:
# GPU Sparsity Comparison - Google Colab
# Dense GPU multiplication at different sparsity levels
# Must match CPU test parameters exactly!

"""
INSTRUCTIONS:
1. Upload this notebook to Google Colab
2. Runtime → Change runtime type → GPU (T4 or better)
3. Upload data files: matrix_a_small.csv, matrix_b_small.csv
4. Run all cells
5. Download gpu_sparsity_results.json

This tests DENSE GPU multiplication at 50%, 90%, 95%, 99% sparsity
to compare with CPU results.
"""

# ============================================================================
# Setup and Imports
# ============================================================================

# Install requirements (if needed)
!pip install numpy scipy cupy-cuda11x -q

import numpy as np
import cupy as cp
import time
import json
from scipy import sparse as sp
import csv

print("GPU Available:", cp.cuda.is_available())
print("GPU Device:", cp.cuda.Device())

# ============================================================================
# Data Loading Functions
# ============================================================================

def load_sparse_matrix_from_csv(filepath, size=1000):
    """Load sparse matrix from CSV (1-based indexing)."""
    rows, cols, vals = [], [], []
    
    with open(filepath, 'r') as f:
        reader = csv.reader(f)
        for parts in reader:
            if len(parts) == 3:
                try:
                    # Convert 1-based to 0-based
                    r = int(parts[0]) - 1
                    c = int(parts[1]) - 1
                    v = int(parts[2])
                    rows.append(r)
                    cols.append(c)
                    vals.append(v)
                except ValueError:
                    continue
    
    # Create scipy sparse matrix, then convert to dense
    sparse_mat = sp.csr_matrix((vals, (rows, cols)), shape=(size, size))
    dense_mat = sparse_mat.toarray()
    
    return dense_mat

def generate_sparse_matrix(size, sparsity_percent, seed):
    """Generate sparse matrix matching CPU test parameters."""
    np.random.seed(seed)
    
    total_elements = size * size
    density = (100 - sparsity_percent) / 100.0
    num_entries = int(total_elements * density)
    
    rows = np.random.randint(0, size, size=num_entries)
    cols = np.random.randint(0, size, size=num_entries)
    values = np.random.randint(1, 11, size=num_entries)  # Integer 1-10
    
    # Create sparse then convert to dense
    sparse_mat = sp.csr_matrix((values, (rows, cols)), shape=(size, size))
    dense_mat = sparse_mat.toarray()
    
    return dense_mat

# ============================================================================
# GPU Benchmark Function
# ============================================================================

def benchmark_gpu_multiplication(A_cpu, B_cpu, num_runs=3):
    """
    Benchmark dense GPU matrix multiplication.
    
    Args:
        A_cpu: numpy array (CPU)
        B_cpu: numpy array (CPU)
        num_runs: number of runs for averaging
    
    Returns:
        avg_time, std_time
    """
    # Transfer to GPU
    A_gpu = cp.asarray(A_cpu)
    B_gpu = cp.asarray(B_cpu)
    
    times = []
    for i in range(num_runs):
        cp.cuda.Stream.null.synchronize()  # Ensure GPU ready
        
        start = time.perf_counter()
        C_gpu = cp.matmul(A_gpu, B_gpu)
        cp.cuda.Stream.null.synchronize()  # Wait for completion
        end = time.perf_counter()
        
        times.append(end - start)
        print(f"  Run {i+1}: {times[-1]:.6f}s")
    
    avg_time = np.mean(times)
    std_time = np.std(times)
    
    return avg_time, std_time

# ============================================================================
# Run Sparsity Tests
# ============================================================================

def run_sparsity_tests():
    """Run GPU tests matching CPU benchmark parameters."""
    
    print("="*70)
    print("GPU SPARSITY COMPARISON - Dense GPU Multiplication")
    print("="*70)
    print()
    
    size = 1000
    sparsity_levels = [50, 90, 95, 99]  # Match CPU tests
    num_runs = 3
    
    results = []
    
    for sparsity in sparsity_levels:
        print(f"\n{'='*70}")
        print(f"Testing: {size}×{size} matrix, {sparsity}% sparsity")
        print(f"{'='*70}")
        
        # Generate matrices (same seeds as CPU tests)
        print("Generating matrices...")
        A_cpu = generate_sparse_matrix(size, sparsity, seed=42)
        B_cpu = generate_sparse_matrix(size, sparsity, seed=123)
        
        nnz_A = np.count_nonzero(A_cpu)
        nnz_B = np.count_nonzero(B_cpu)
        actual_sparsity = 100 * (1 - nnz_A / (size * size))
        
        print(f"Matrix A: {nnz_A:,} non-zeros ({actual_sparsity:.2f}% sparse)")
        print(f"Matrix B: {nnz_B:,} non-zeros")
        
        # Benchmark GPU
        print(f"\nBenchmarking DENSE GPU (cupy.matmul)...")
        gpu_time, gpu_std = benchmark_gpu_multiplication(A_cpu, B_cpu, num_runs)
        print(f"Average: {gpu_time:.6f}s ± {gpu_std:.6f}s")
        
        results.append({
            "sparsity_percent": sparsity,
            "matrix_size": size,
            "nnz_A": int(nnz_A),
            "nnz_B": int(nnz_B),
            "actual_sparsity": float(actual_sparsity),
            "gpu_time": float(gpu_time),
            "gpu_std": float(gpu_std),
            "method": "Dense GPU (cupy)"
        })
        
        print(f"{'='*70}")
    
    return results

# ============================================================================
# Main Execution
# ============================================================================

if __name__ == "__main__":
    # Run tests
    results = run_sparsity_tests()
    
    # Print summary
    print("\n\n" + "="*70)
    print("SUMMARY - GPU Results")
    print("="*70)
    
    for r in results:
        print(f"{r['sparsity_percent']}% sparse: {r['gpu_time']:.6f}s")
    
    # Save results
    output = {
        "gpu_device": str(cp.cuda.Device()),
        "test_type": "sparsity_comparison",
        "matrix_size": 1000,
        "results": results
    }
    
    with open('gpu_sparsity_results.json', 'w') as f:
        json.dump(output, f, indent=2)
    
    print("\n✓ Results saved to gpu_sparsity_results.json")
    print("Download this file and copy to google_colab_gpu/results/")
    
    # Optional: Display results for manual copy
    print("\n" + "="*70)
    print("GPU TIMES (for manual comparison):")
    print("="*70)
    for r in results:
        print(f"Sparsity {r['sparsity_percent']}%: {r['gpu_time']:.6f}s ± {r['gpu_std']:.6f}s")